In [1]:
# %% [markdown]
# # Urban Heat Island Prediction with Satellite Data
# ## Optimized Pipeline with Gradient Boosting

# %% [code]
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# %% [code]
# Import libraries
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import rasterio
from rasterio import transform
from pystac_client import Client
import planetary_computer
from shapely.geometry import Polygon
from datetime import datetime
from tqdm import tqdm
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
import matplotlib.pyplot as plt

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# %% [code]
# File paths
TRAIN_PATH = '/content/drive/MyDrive/Colab Notebooks/Training_data_uhi_index_UHI2025-v3.csv'
TEST_PATH = '/content/drive/MyDrive/Colab Notebooks/Submission_template_UHI2025-v3.csv'

# %% [code]
# Load data
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

# Convert datetime
def convert_date(date_str):
    return datetime.strptime(date_str[:10], "%d-%m-%Y").strftime("%Y-%m-%d")

train_df['datetime'] = train_df['datetime'].apply(convert_date)

# %% [code]
# STAC API configuration
STAC_API = "https://planetarycomputer.microsoft.com/api/stac/v1"
BANDS = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 
         'B07', 'B08', 'B8A', 'B11', 'B12']

# %% [code]
def fetch_satellite_data(df):
    """
    Fetch satellite data for all points in dataframe
    Returns xarray Dataset with all bands
    """
    catalog = Client.open(STAC_API)
    
    all_data = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        # Create point geometry
        geometry = Polygon.from_bounds(
            row['Longitude'], row['Latitude'],
            row['Longitude'], row['Latitude']
        )
        
        # Search for items
        search = catalog.search(
            collections=["sentinel-2-l2a"],
            datetime=row['datetime'],
            query={"eo:cloud_cover": {"lt": 10}},
            intersects=geometry
        )
        
        items = list(search.get_items())
        if not items:
            continue
            
        # Take the least cloudy scene
        best_item = min(items, key=lambda x: x.properties['eo:cloud_cover'])
        signed_item = planetary_computer.sign(best_item)
        
        # Load data
        data = stac_load(
            [signed_item],
            bands=BANDS,
            crs="EPSG:4326",
            resolution=0.00027,  # ~10m resolution
            chunks={},
            bbox=[row['Longitude'], row['Latitude'], row['Longitude'], row['Latitude']]
        ).compute()
        
        all_data.append(data)
    
    return xr.concat(all_data, dim='time')

# %% [code]
# Fetch training satellite data
print("Fetching training satellite data...")
train_sat_data = fetch_satellite_data(train_df)

# %% [code]
# Feature engineering functions
def calculate_indices(ds):
    """Calculate spectral indices from satellite data"""
    ds['NDVI'] = (ds.B08 - ds.B04) / (ds.B08 + ds.B04)
    ds['NDBI'] = (ds.B11 - ds.B08) / (ds.B11 + ds.B08)
    ds['NDWI'] = (ds.B03 - ds.B08) / (ds.B03 + ds.B08)
    ds['SAVI'] = 1.5 * (ds.B08 - ds.B04) / (ds.B08 + ds.B04 + 0.5)
    ds['MSAVI'] = (2 * ds.B08 + 1 - 
                  np.sqrt((2 * ds.B08 + 1)**2 - 8 * (ds.B08 - ds.B04))) / 2
    ds['UI'] = (ds.B12 - ds.B08) / (ds.B12 + ds.B08)
    return ds

# %% [code]
# Process training data
print("Processing training data...")
train_sat_data = calculate_indices(train_sat_data)
features = train_sat_data.to_dataframe().reset_index()

# Merge with original dataframe
full_train_df = pd.merge(
    train_df,
    features,
    left_on=['datetime', 'Latitude', 'Longitude'],
    right_on=['time', 'latitude', 'longitude']
)

# Clean data
full_train_df = full_train_df.dropna()
X = full_train_df.drop(columns=['UHI Index', 'datetime', 'time', 'latitude', 'longitude'])
y = full_train_df['UHI Index']

# %% [code]
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# %% [code]
# Hyperparameter tuning
params = {
    'n_estimators': 500,
    'learning_rate': 0.05,
    'max_depth': 7,
    'subsample': 0.8,
    'colsample_bytree': 0.9,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1
}

# Initialize and train model
model = lgb.LGBMRegressor(**params)
model.fit(X_train_scaled, y_train,
          eval_set=[(X_test_scaled, y_test)],
          early_stopping_rounds=50,
          verbose=100)

# Evaluate model
train_pred = model.predict(X_train_scaled)
test_pred = model.predict(X_test_scaled)

print(f"Train R²: {r2_score(y_train, train_pred):.4f}")
print(f"Test R²: {r2_score(y_test, test_pred):.4f}")

# Cross-validation
cv_scores = cross_val_score(model, X_train_scaled, y_train, 
                            cv=5, scoring='r2')
print(f"Cross-validation R²: {np.mean(cv_scores):.4f} ± {np.std(cv_scores):.4f}")

# %% [code]
# Process test data
print("Processing test data...")
test_sat_data = fetch_satellite_data(test_df)
test_sat_data = calculate_indices(test_sat_data)
test_features = test_sat_data.to_dataframe().reset_index()

# Prepare submission
test_scaled = scaler.transform(test_features[X.columns])
test_pred = model.predict(test_scaled)

# Create submission file
submission_df = pd.DataFrame({
    'Longitude': test_df['Longitude'],
    'Latitude': test_df['Latitude'],
    'UHI Index': test_pred
})

# Save results
submission_df.to_csv('/content/drive/MyDrive/Colab Notebooks/UHI_Final_Submission.csv', index=False)
print("Submission file saved!")

# %% [code]
# Feature importance visualization
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 6))
plt.barh(feature_importance['Feature'], feature_importance['Importance'])
plt.title('Feature Importance')
plt.xlabel('Importance Score')
plt.show()

ModuleNotFoundError: No module named 'google.colab'